# Solving Burgers' Equation Using PINNs

### Problem Statement
We aim to solve the **Burgers' equation** using **Physics-Informed Neural Networks (PINNs)**. ### Context: What is Burgers' Equation and Why is it Useful?

The **Burgers' equation** is a fundamental partial differential equation (PDE) that appears in various fields of applied mathematics, physics, and engineering. It is expressed as:

$$
u_t + u u_x = \nu u_{xx},
$$

where:
- $u(t, x)$ is the unknown function,
- $u_t$ represents the temporal derivative,
- $u_x$ is the spatial derivative,
- $u_{xx}$ is the second spatial derivative,
- $\nu$ is the viscosity term (a positive constant).

Burgers' equation is particularly useful because it models a wide range of physical phenomena, including **nonlinear wave propagation**, **shock waves**, **turbulence**, and **heat conduction**. It combines two key effects:
1. **Nonlinear advection** (through the $ u u_x $ term), which governs the steepening of waves.
2. **Linear diffusion** (through the $ \nu u_{xx} $ term), which smoothens sharp gradients.

This combination of effects makes Burgers' equation an important test case for studying the interplay between nonlinearity and diffusion in complex systems.

In practice, Burgers' equation is often used as a benchmark problem to validate new **numerical methods**, **machine learning techniques**, and **data-driven approaches**. Known analytical solutions for specific cases provide a reference for comparing approximate solutions, such as those computed using **Physics-Informed Neural Networks (PINNs)**. Solving Burgers' equation helps researchers explore broader phenomena in **fluid dynamics**, **traffic flow**, and **acoustic waves**, while also assessing the effectiveness of computational techniques.

The initial and boundary conditions are:

- **Initial Condition**:
  $$
  u(0, x) = -\sin(\pi x), \quad x \in [-1, 1].
  $$
- **Boundary Conditions**:
  $$
  u(t, -1) = u(t, 1) = 0, \quad t \in [0, 1].
  $$

The goal is to approximate $u(t, x)$ using a PINN over the domain $t \in [0, 1], x \in [-1, 1]$.

In [1]:
# Import required libraries
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

## Neural Network Architecture
PINNs use a neural network to represent the solution \( u(t, x) \). The network takes the inputs \( t \) and \( x \) and outputs \( u(t, x) \). Here's the structure:
- Input layer: 2 features (\( t \) and \( x \))
- Hidden layers: 4 fully connected layers with 50 neurons each
- Activation function: `tanh` for all hidden layers
- Output layer: A single scalar representing \( u(t, x) \)

In [2]:
# Define the neural network architecture
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.input_layer = nn.Linear(2, layers[0])
        self.hidden_layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.hidden_layers.append(nn.Linear(layers[i], layers[i + 1]))
        self.output_layer = nn.Linear(layers[-1], 1)
        self.activation = nn.Tanh()

    def forward(self, t, x):
        input_tensor = torch.cat([t, x], dim=1)
        input_tensor = self.activation(self.input_layer(input_tensor))
        for layer in self.hidden_layers:
            input_tensor = self.activation(layer(input_tensor))
        return self.output_layer(input_tensor)

## Physics-Informed Loss Function
The PINN minimizes two loss terms:
1. **Data Loss**:
   ```math
   \text{Loss}_u = \frac{1}{N_u} \sum_{i=1}^{N_u} \left( u_i - u_\text{pred}(t_i, x_i) \right)^2
   ```
2. **PDE Residual Loss**:
   ```math
   \text{Loss}_f = \frac{1}{N_f} \sum_{i=1}^{N_f} f(t_i, x_i)^2,
   ```
   where:
   ```math
   f(t, x) = u_t + u u_x - \nu u_{xx}.
   ```

In [3]:
# Define the physics-informed loss function
def pde_loss(model, t, x):
    t.requires_grad_(True)
    x.requires_grad_(True)

    u = model(t, x)

    # Compute partial derivatives
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

    # Define the PDE residual
    f = u_t + u * u_x - (0.01 / np.pi) * u_xx
    return f

## Create Training Data
To train the PINN, we need:
- Initial condition data: \( u(0, x) = -\sin(\pi x) \)
- Boundary condition data: \( u(t, -1) = u(t, 1) = 0 \)
- Collocation points: Randomly sample \( t \in [0, 1] \) and \( x \in [-1, 1] \)

In [4]:
def create_training_data():
    # Initial condition
    x = np.linspace(-1, 1, 256).reshape(-1, 1)
    t = np.zeros_like(x)
    u = -np.sin(np.pi * x)

    # Boundary conditions
    t_boundary = np.linspace(0, 1, 100).reshape(-1, 1)
    x_left = -np.ones_like(t_boundary)
    x_right = np.ones_like(t_boundary)
    u_left = np.zeros_like(t_boundary)
    u_right = np.zeros_like(t_boundary)

    # Collocation points
    t_f = np.random.uniform(0, 1, 10000).reshape(-1, 1)
    x_f = np.random.uniform(-1, 1, 10000).reshape(-1, 1)

    # Convert all to PyTorch tensors
    t_u = torch.tensor(t, dtype=torch.float32)
    x_u = torch.tensor(x, dtype=torch.float32)
    u = torch.tensor(u, dtype=torch.float32)
    t_boundary = torch.tensor(t_boundary, dtype=torch.float32)
    x_left = torch.tensor(x_left, dtype=torch.float32)
    x_right = torch.tensor(x_right, dtype=torch.float32)
    u_left = torch.tensor(u_left, dtype=torch.float32)
    u_right = torch.tensor(u_right, dtype=torch.float32)
    t_f = torch.tensor(t_f, dtype=torch.float32)
    x_f = torch.tensor(x_f, dtype=torch.float32)

    return t_u, x_u, u, t_boundary, x_left, x_right, u_left, u_right, t_f, x_f

## Training the PINN
We combine the data loss and PDE residual loss to train the PINN.

In [5]:
# Define the training loop
def train(model, optimizer, t_u, x_u, u, t_f, x_f, epochs):
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Data loss
        u_pred = model(t_u, x_u)
        loss_u = torch.mean((u - u_pred) ** 2)

        # PDE residual loss
        f_pred = pde_loss(model, t_f, x_f)
        loss_f = torch.mean(f_pred ** 2)

        # Total loss
        loss = loss_u + loss_f
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}, Loss_u: {loss_u.item()}, Loss_f: {loss_f.item()}")

## Visualize Results
After training, we plot the predicted solution and the loss function.

In [6]:
# Plotting function
def plot_results(model):
    t = np.linspace(0, 1, 100)
    x = np.linspace(-1, 1, 256)
    T, X = np.meshgrid(t, x)

    t_test = torch.tensor(T.flatten().reshape(-1, 1), dtype=torch.float32)
    x_test = torch.tensor(X.flatten().reshape(-1, 1), dtype=torch.float32)
    u_pred = model(t_test, x_test).detach().numpy().reshape(256, 100)

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred, extent=[0, 1, -1, 1], origin='lower', aspect='auto', cmap='jet')
    plt.colorbar(label='u(t, x)')
    plt.xlabel('t')
    plt.ylabel('x')
    plt.title('Predicted solution u(t, x)')
    plt.show()

In [ ]:
# Main execution
if __name__ == "__main__":
    layers = [50, 50, 50, 50]
    model = PINN(layers)

    t_u, x_u, u, t_boundary, x_left, x_right, u_left, u_right, t_f, x_f = create_training_data()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train(model, optimizer, t_u, x_u, u, t_f, x_f, epochs=5000)
    plot_results(model)

Epoch 0, Loss: 0.5301456451416016, Loss_u: 0.52680504322052, Loss_f: 0.0033406266011297703
Epoch 100, Loss: 0.14273175597190857, Loss_u: 0.0984220877289772, Loss_f: 0.04430966079235077
Epoch 200, Loss: 0.11678905785083771, Loss_u: 0.07471741735935211, Loss_f: 0.042071644216775894
Epoch 300, Loss: 0.09481151401996613, Loss_u: 0.060708366334438324, Loss_f: 0.0341031476855278
Epoch 400, Loss: 0.0851333737373352, Loss_u: 0.052189841866493225, Loss_f: 0.03294353559613228
Epoch 500, Loss: 0.0776607096195221, Loss_u: 0.04729670286178589, Loss_f: 0.030364006757736206
Epoch 600, Loss: 0.0673774927854538, Loss_u: 0.04164574667811394, Loss_f: 0.025731749832630157
Epoch 700, Loss: 0.05717775970697403, Loss_u: 0.034085530787706375, Loss_f: 0.023092228919267654
Epoch 800, Loss: 0.05265479534864426, Loss_u: 0.027640676125884056, Loss_f: 0.02501412108540535
Epoch 900, Loss: 0.06280390918254852, Loss_u: 0.023775428533554077, Loss_f: 0.039028480648994446
Epoch 1000, Loss: 0.02946292981505394, Loss_u: 0.